# Laser Power Combiner & Plotting

This notebook was converted from `20250916_laserpower_combiner_plotting.py`.
It provides functions to parse instrument CSVs ("Result table values"),
combine measurements by wavelength, export combined sheets, and plot results.

Usage: set `input_folder`, `output_excel`, and `plot_folder`, then run the final cell to execute the pipeline.

## Overview

- `read_power_instruction_table(file_path)`: parses a CSV-like text file and returns a DataFrame of the "Result table values" section.
- `combine_group(dfs_dict, wavelength)`: merges multiple DataFrames (same wavelength) into one table keyed by power percentage.
- `plot_wavelength_data(combined_all, save_folder)`: saves errorbar plots per wavelength.
- `main(input_folder, output_excel, plot_folder)`: runs the end-to-end pipeline.

In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import re

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Function: parse 'Result table values' section into a DataFrame
def read_power_instruction_table(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Find start of 'Result table values'
    start_idx = None
    for i, line in enumerate(lines):
        if 'Result table values' in line:
            start_idx = i + 1
            break
    if start_idx is None:
        raise ValueError(f"No 'Result table values' found in {file_path}")

    header = lines[start_idx].strip().split(';')

    data = []
    for line in lines[start_idx+1:]:
        if line.strip().lower().startswith('time'):
            break
        if not line.strip():
            break
        data.append(line.strip().split(';'))

    df = pd.DataFrame(data, columns=header)
    df = df.rename(columns={'power_instruction': 'power_percentage_values'})
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except (ValueError, TypeError):
            pass
    return df

In [2]:
# Combine multiple files (same wavelength) into one DataFrame keyed by power%
def combine_group(dfs_dict, wavelength):
    combined = None
    for fname, df in dfs_dict.items():
        date = fname.split('_')[0]
        cols = [c for c in df.columns if c != 'power_percentage_values']
        rename_dict = {c: f"{date}_{c}" for c in cols}
        df_renamed = df.rename(columns=rename_dict)
        if combined is None:
            combined = df_renamed
        else:
            combined = pd.merge(combined, df_renamed, on='power_percentage_values', how='outer')
    if combined is not None:
        combined = (combined.drop_duplicates(subset=['power_percentage_values']).sort_values('power_percentage_values'))
    return combined

# Plotting helper
def plot_wavelength_data(combined_all, save_folder='plots'):
    os.makedirs(save_folder, exist_ok=True)
    for wl, df in combined_all.items():
        plt.figure(figsize=(8, 6))
        month_map = {}
        for col in df.columns:
            if col == 'power_percentage_values':
                continue
            if '_power' in col:
                month = col.split('_')[0]
                month_num = int(month.split('-')[0])
                month_map[month] = month_num
        for month in sorted(month_map, key=lambda x: month_map[x]):
            power_col = [c for c in df.columns if c.startswith(month) and '_power' in c][0]
            error_col = [c for c in df.columns if c.startswith(month) and '_error' in c][0]
            plt.errorbar(df['power_percentage_values'], df[power_col], yerr=df[error_col].abs(), marker='o', capsize=3, label=f"{month}")
        plt.title(f"Laser Power Calibration - {wl}nm")
        plt.xlabel('Power Percentage Values')
        plt.ylabel('Measured Power (mW)')
        plt.legend(title='Month')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plot_file = os.path.join(save_folder, f"laser_power_{wl}nm.png")
        plt.savefig(plot_file, dpi=300)
        plt.close()
        print(f"✅ Saved plot for {wl}nm → {plot_file}")

In [3]:
# Main pipeline
def main(input_folder, output_excel, plot_folder):
    files = glob.glob(os.path.join(input_folder, '*.csv'))
    if not files:
        print('⚠️ No CSV files found in folder.')
        return
    dfs = {os.path.basename(file): read_power_instruction_table(file) for file in files}
    grouped = {}
    for fname, df in dfs.items():
        match = re.search(r'_(\d+)\.csv', fname)
        if match:
            wl = match.group(1)
            grouped.setdefault(wl, {})[fname] = df
    combined_all = {wl: combine_group(dfs_dict, wl) for wl, dfs_dict in grouped.items()}
    try:
        import openpyxl
        if combined_all:
            with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
                for wl in sorted(combined_all.keys(), key=lambda x: int(x)):
                    df = combined_all[wl]
                    if df is not None and not df.empty:
                        df.to_excel(writer, sheet_name=f"{wl}nm", index=False)
            print(f"✅ Exported all wavelengths (sorted) to {output_excel}")
        else:
            print('⚠️ No data found to export.')
    except ImportError:
        print('⚠️ openpyxl not installed. Exporting as separate CSV files instead...')
        for wl in sorted(combined_all.keys(), key=lambda x: int(x)):
            df = combined_all[wl]
            if df is not None and not df.empty:
                csv_file = f"combined_{wl}nm.csv"
                df.to_csv(csv_file, index=False)
                print(f"✅ Saved {csv_file}")
    plot_wavelength_data(combined_all, save_folder=plot_folder)

## Run the pipeline

Set your paths below and run the cell to execute the pipeline. Adjust `input_folder` to your CSV directory.

Example:

In [4]:
# Prompt for input folder and ask for output paths
import os

cwd_string = os.getcwd()
print("Current Working Directory:", cwd_string)

input_folder = os.path.join(cwd_string, 'input_data')
print(input_folder)

output_dir = os.path.join(cwd_string, 'output_data')
os.makedirs(output_dir, exist_ok=True)

plot_folder = output_dir
print(plot_folder)

output_excel = os.path.join (output_dir, 'combined_power_data.xlsx')
print(output_excel)

# To run the pipeline uncomment the next line
main(input_folder, output_excel, plot_folder)

Current Working Directory: /Users/narasa01/Documents/code/github/QA_measurements
/Users/narasa01/Documents/code/github/QA_measurements/input_data


NameError: name 'plot_folder' is not defined